In [1]:
import json, requests
import os
import numpy as np
import pandas as pd

In [2]:
API_KEY = 'JRLUCAS'

In [3]:
def get_price_history(symbols):
    candle_list = []
    df_dict = {}
    for symbol in symbols:
        API_URL = r'https://api.tdameritrade.com/v1/marketdata/{}/pricehistory'.format(symbol)
        params = {
            'apikey': API_KEY,
            'periodType': 'year',
            'period': 5,
            'frequencyType': 'daily',
            'frequency': 1,
            'needExtendedHoursData': 'false'
        }

        content = requests.get(url=(API_URL),params=params).text

        data = json.loads(content)
        candle_list.append(data)
        for i in candle_list:
            df = pd.DataFrame.from_dict(i['candles'])
            pickle_name = symbol
            pickle_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', pickle_name + '.pickle'))
            df.to_pickle(pickle_path)

In [4]:
etf_list = ['VTI','VEA', 'VWO', 'VNQ', 'XLE', 'BND', 'SCHP', 'VTEB', 'VIG']

In [5]:
get_price_history(etf_list)